Welcome to my notebook where we build a recommendation engine on the movie lens dataset

In [ ]:
!pip install pyspark

The MovieLens 100K dataset contains 100,000 ratings from 943 users on 1,682 movies, provided by GroupLens Research. This dataset includes multiple files that organize and split data for recommendation system experiments.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, round, to_date
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
os.chdir('ml-100k/')

In [ ]:
spark = SparkSession.builder \
    .appName("MySparkSession") \
    .getOrCreate()

In [ ]:
# Return spark version
print(spark.version)

# Return python version
import sys
print(sys.version_info)

3.5.3
sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", StringType(), True)
])

ratings = spark.read.csv("u.data", sep="\t", schema=schema)
ratings.show()


+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|   3.0|881250949|
|    186|     302|   3.0|891717742|
|     22|     377|   1.0|878887116|
|    244|      51|   2.0|880606923|
|    166|     346|   1.0|886397596|
|    298|     474|   4.0|884182806|
|    115|     265|   2.0|881171488|
|    253|     465|   5.0|891628467|
|    305|     451|   3.0|886324817|
|      6|      86|   3.0|883603013|
|     62|     257|   2.0|879372434|
|    286|    1014|   5.0|879781125|
|    200|     222|   5.0|876042340|
|    210|      40|   3.0|891035994|
|    224|      29|   3.0|888104457|
|    303|     785|   3.0|879485318|
|    122|     387|   5.0|879270459|
|    194|     274|   2.0|879539794|
|    291|    1042|   4.0|874834944|
|    234|    1184|   2.0|892079237|
+-------+--------+------+---------+
only showing top 20 rows



typically users will be given an ID but items or in this case movies would either have an id or a name and need to be given an ID

In [ ]:
ratings.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|   3.0|881250949|
|    186|     302|   3.0|891717742|
|     22|     377|   1.0|878887116|
|    244|      51|   2.0|880606923|
|    166|     346|   1.0|886397596|
+-------+--------+------+---------+
only showing top 5 rows



In [ ]:
# next we look at sparsity
numerator = ratings.count()
users = ratings.select('user_id').distinct().count()
items = ratings.select('movie_id').distinct().count()
denominator = users * items
sparsity = 1 - (numerator * 1.0 / denominator)
print("The ratings dataframe is ", "%.2f" % (sparsity*100), "% empty.")

The ratings dataframe is  93.70 % empty.


# Lets go through ALS hyper parameters

k number off latent features attempted to be uncovered this can be interchanged and experimented with cross validation can help here
k= 10

mamiter just assigns the maximum times of going back and forth adjustinng the matrices

regParam also known as lambda is the function to reduce overfitting

alpha is used on implicit ratings and not explicit

Nonnegative = True  this is the factorisation method we are using

ColdStartStraegy tells the model when calculating the error metric to ignore the values where the user isn’t in both training and test

als_model =(userCol=‘userId’, itemCol=‘movieId’, ratingCol=‘rating, rank=k, maxIter=100, alpha=40,nonnegative=True, coldStartStrategy=True,implicitPrefs=False)

In [ ]:
# Split the ratings dataframe into training and test data
(training_data, test_data) = ratings.randomSplit([0.8, 0.2], seed=42)

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
# Set the ALS hyperparameters
# Initialize ALS model
als = ALS(
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop",  # Avoid NaN in predictions
    nonnegative=True,
    implicitPrefs=False  # For explicit ratings
)

In [ ]:

# Define the evaluation metric (RMSE)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

In [ ]:

# Build a hyperparameter grid for tuning
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 20]) \
    .addGrid(als.maxIter, [10, 15, 20]) \
    .addGrid(als.regParam, [0.01, 0.1, 1.0]) \
    .build()

# Set up cross-validator
crossval = CrossValidator(
    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3  # Number of folds for cross-validation
)

In [ ]:
# Fit the model using cross-validation
cvModel = crossval.fit(training_data)

# Get the best model
best_model = cvModel.bestModel

# Generate predictions on the test data
test_predictions = best_model.transform(test_data)

# Evaluate the model's performance on the test set
rmse = evaluator.evaluate(test_predictions)

In [ ]:
mae_evaluator = RegressionEvaluator(
    metricName="mae",
    labelCol="rating",
    predictionCol="prediction"
)

# Calculate MAE on the test predictions
mae = mae_evaluator.evaluate(test_predictions)
print(f"Test MAE: {mae}")

total_items = ratings.select("movie_id").distinct().count()

# Calculate the unique items covered by the predictions
covered_items = test_predictions.select("movie_id").distinct().count()

# Compute coverage as a percentage
coverage = (covered_items / total_items) * 100
print(f"Coverage: {coverage:.2f}%")

# Print the best hyperparameters and RMSE
print(f"Best Rank: {best_model._java_obj.parent().getRank()}")
print(f"Best MaxIter: {best_model._java_obj.parent().getMaxIter()}")
print(f"Best RegParam: {best_model._java_obj.parent().getRegParam()}")
print(f"Test RMSE: {rmse}")

# Show predictions
test_predictions.show()

Test MAE: 0.7212871997108401
Coverage: 81.39%
Best Rank: 5
Best MaxIter: 20
Best RegParam: 0.1
Test RMSE: 0.911226931533807
+-------+--------+------+---------+----------+
|user_id|movie_id|rating|timestamp|prediction|
+-------+--------+------+---------+----------+
|    148|       8|   4.0|877020297| 3.6209981|
|    148|      56|   5.0|877398212|  4.118529|
|    148|      71|   5.0|877019251|  3.267142|
|    148|     133|   5.0|877019251| 3.1141782|
|    148|     169|   5.0|877020297| 4.5899434|
|    148|     172|   5.0|877016513| 4.4839625|
|    148|     194|   5.0|877015066|   3.93477|
|    148|     222|   4.0|877398901|  3.586647|
|    148|     357|   5.0|877016735|   4.10935|
|    148|     432|   5.0|877019698|  3.601791|
|    148|     474|   5.0|877019882| 4.1769524|
|    148|     529|   5.0|877398901| 3.5655146|
|    148|     588|   4.0|877399018| 3.3127081|
|    148|     713|   3.0|877021535| 3.2538805|
|    463|      24|   3.0|877385731| 2.2830355|
|    463|      50|   4.0|89053

1. Model Accuracy (MAE and RMSE):
MAE (Mean Absolute Error):
An MAE of 0.72 means that, on average, the absolute difference between the predicted ratings and the actual ratings is 0.72 on a rating scale (1 to 5).
This suggests relatively small average deviations, which is a good indicator of accuracy.
RMSE (Root Mean Squared Error):
An RMSE of 0.92 is slightly higher than the MAE. RMSE penalizes larger errors more than MAE does because it squares the differences.
The slightly higher RMSE suggests the presence of some predictions that are farther off from the true values, but these errors are not overwhelmingly large.
Takeaway: The errors (0.72 for MAE and 0.92 for RMSE) suggest the model is reasonably accurate, with most predictions being close to the actual values, though there are a few outliers with larger deviations.

2. Coverage (82%):
Coverage refers to the percentage of unique items (movies) in the dataset for which the model generated predictions.

High coverage suggests the model generalizes well to a broad range of items.
Missing 18% of the movies might indicate sparse data for those movies or user-movie combinations that the model couldn't predict effectively.

3. Overall Model Performance:
The combination of reasonable accuracy (MAE and RMSE) and high coverage (82%) suggests a balanced model.
